In [ ]:
# Copyright 2021 NVIDIA CORPORATION
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
n_files = 50

# Apply TE and other features

In [2]:
import numpy as np
import glob
import gc
import cudf
import gc

In [3]:
TE_files = sorted(glob.glob('/raid/TE_opt/*.parquet'))

In [4]:
TE_files

['/raid/TE_opt/a_user_id.parquet',
 '/raid/TE_opt/b_is_verified_tweet_type.parquet',
 '/raid/TE_opt/b_user_id.parquet',
 '/raid/TE_opt/b_user_id_a_user_id.parquet',
 '/raid/TE_opt/b_user_id_tweet_type_language.parquet',
 '/raid/TE_opt/domains_language_b_follows_a_tweet_type_media_a_is_verified.parquet',
 '/raid/TE_opt/media_tweet_type_language.parquet',
 '/raid/TE_opt/media_tweet_type_language_a_is_verified_b_is_verified_b_follows_a.parquet',
 '/raid/TE_opt/tw_original_user0_tweet_type_language.parquet',
 '/raid/TE_opt/tw_original_user1_tweet_type_language.parquet',
 '/raid/TE_opt/tweet_type.parquet']

In [5]:
!rm -r /raid/recsys2021_pre_TE/
!mkdir /raid/recsys2021_pre_TE/

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [6]:
DONT_USE = ['a_account_creation','b_account_creation','engage_time',
            'fold', 'dt_dow', 'a_account_creation', 
            'b_account_creation', 'elapsed_time', 'links','domains','hashtags','id', 'date', 'is_train', 
            'tw_original_http0', 'tw_original_user0', 'tw_original_user1', 'tw_original_user2',
            'tw_rt_count_char', 'tw_rt_count_words', 'tw_rt_user0', 'tw_tweet', 'tw_word0', 'a_user_id_x', 'b_user_id_x',
            'tw_word1', 'tw_word2', 'tw_word3', 'tw_word4', 'tw_count_hash', 'dt_minute', 'dt_second', 'dt_day', 'id']

In [7]:
VALID_DOW = '2021-02-18'
psmooth=20
means = {}
means['reply'] = 0.02846728456689906
means['like'] = 0.3968895210408169
means['retweet'] = 0.08769760903336701
means['retweet_comment'] = 0.006918407917391091

def add_TE(fn):
    df = cudf.read_parquet(fn)
    df['date'] = cudf.to_datetime(df['timestamp'], unit='s')
    df['dt_dow']  = df['date'].dt.weekday
    df['dt_hour'] = df['date'].dt.hour
    df['dt_minute'] = df['date'].dt.minute
    df['dt_second'] = df['date'].dt.second
    if VALID_DOW!='':
        df['is_train'] = (df['date']<cudf.to_datetime(VALID_DOW)).astype(np.int8)
    else:
        df['is_train'] = 1
    for i, file in enumerate(TE_files):
        df_tmp = cudf.read_parquet(file)
        col = [x for x in df_tmp.columns if not('reply' in x or ('retweet' in x and 'tw_len_retweet' not in x) or '_retweet_comment' in x or 'like' in x)]
        col_rest = [x for x in df_tmp.columns if x not in col]
        df = df.merge(df_tmp, on=col, how='left')
        for key in means.keys():
            if df_tmp.shape[0]>1000:
                df['TE_' + '_'.join(col) + '_' + key] = (((df[key + '_sum']-df[key]*df['is_train'])+means[key]*psmooth)/(df['reply_count']-df['is_train']+psmooth))
            else:
                df['TE_' + '_'.join(col) + '_' + key] = (((df[key + '_sum'])+means[key]*psmooth)/(df['reply_count']+psmooth))
            if col[0]=='a_user_id' and key=='like':
                df.loc[df['reply_count']<=1000, 'TE_' + '_'.join(col) + '_' + key] = None
            df['TE_' + '_'.join(col) + '_' + key] = df['TE_' + '_'.join(col) + '_' + key].fillna(np.float32(means[key])).round(3)
        df.drop(col_rest, inplace=True, axis=1)
        col = [x for x in df_tmp.columns if not('reply' in x or ('retweet' in x and 'tw_len_retweet' not in x) or '_retweet_comment' in x or 'like' in x)]
        if 'b_user_id' not in col and 'a_user_id' in col:
            print(col)
            dfcols = list(df_tmp.columns)
            dfcolsnew = []
            for col in dfcols:
                if col == 'b_user_id':
                    dfcolsnew.append('a_user_id')
                elif col == 'a_user_id':
                    dfcolsnew.append('b_user_id')
                else:
                    dfcolsnew.append(col)
            df_tmp.columns = dfcolsnew
            col = [x for x in df_tmp.columns if not('reply' in x or ('retweet' in x and 'tw_len_retweet' not in x) or '_retweet_comment' in x or 'like' in x)]
            col_rest = [x for x in df_tmp.columns if x not in col]
            df = df.merge(df_tmp, on=col, how='left')
            for key in means.keys():
                df['TE_switch_' + '_'.join(col) + '_' + key] = (((df[key + '_sum'])+means[key]*psmooth)/(df['reply_count']+psmooth))
                df['TE_switch_' + '_'.join(col) + '_' + key] = df['TE_switch_' + '_'.join(col) + '_' + key].fillna(np.float32(means[key])).round(3)
            df.drop(col_rest, inplace=True, axis=1)
            del df_tmp
            gc.collect()
    df['a_ff_rate'] = (df['a_following_count'] / (1+df['a_follower_count'])).astype('float32')
    df['b_ff_rate'] = (df['b_follower_count']  / (1+df['b_following_count'])).astype('float32')
    df['ab_fing_rate'] = (df['a_following_count'] / (1+df['b_following_count'])).astype('float32')
    df['ab_fer_rate'] = (df['a_follower_count'] / (1+df['b_follower_count'])).astype('float32')
    df['ab_age_dff'] = (df['a_account_creation']-df['b_account_creation'])
    df['ab_age_rate'] = (df['a_account_creation']+129)/(df['b_account_creation']+129)
    final_cols = [x for x in sorted(list(df.columns)) if x not in DONT_USE]
    df[final_cols].to_parquet( '/raid/recsys2021_pre_TE/' + fn.split('/')[-1])

In [8]:
files = sorted(glob.glob('/raid/recsys2021_pre/*'))[0:n_files]

In [9]:
%%time

for file in files:
    print(file)
    add_TE(file)

/raid/recsys2021_pre/part-00000.parquet
['a_user_id']
/raid/recsys2021_pre/part-00001.parquet
['a_user_id']
/raid/recsys2021_pre/part-00002.parquet
['a_user_id']
/raid/recsys2021_pre/part-00003.parquet
['a_user_id']
/raid/recsys2021_pre/part-00004.parquet
['a_user_id']
/raid/recsys2021_pre/part-00005.parquet
['a_user_id']
/raid/recsys2021_pre/part-00006.parquet
['a_user_id']
/raid/recsys2021_pre/part-00007.parquet
['a_user_id']
/raid/recsys2021_pre/part-00008.parquet
['a_user_id']
/raid/recsys2021_pre/part-00009.parquet
['a_user_id']
/raid/recsys2021_pre/part-00010.parquet
['a_user_id']
/raid/recsys2021_pre/part-00011.parquet
['a_user_id']
/raid/recsys2021_pre/part-00012.parquet
['a_user_id']
/raid/recsys2021_pre/part-00013.parquet
['a_user_id']
/raid/recsys2021_pre/part-00014.parquet
['a_user_id']
/raid/recsys2021_pre/part-00015.parquet
['a_user_id']
/raid/recsys2021_pre/part-00016.parquet
['a_user_id']
/raid/recsys2021_pre/part-00017.parquet
['a_user_id']
/raid/recsys2021_pre/part-00